# Matemática Modular


# Imports


In [1]:
import hashlib

from Crypto.Util import number
from base64 import b64encode, b64decode
from Cryptodome.Random import get_random_bytes
from Cryptodome.Cipher import AES

In [ ]:
encoding = 'utf-8'

sk = ''
pk = ''
g = ''
p = ''
m = ''

# Primeiro passo

In [3]:
bitSize = 1024
p = number.getPrime(bitSize)
g = number.getPrime(bitSize)

p, g

(167030763938411237445083630386299604864905183265564095664089099126200347051912112160569412698264579128581046528174528856895064059238428841076918894220752177842044435713826039285198886773573688521913119147898652220951299519686773130443080885782966223268358388008280396286792297275477384183350163904836460648361,
 108093252116066056494208390429822377003084108096214879042943192438971288699660259944538779168961587716703974203654611567031123323015391332964726393670988082537034623442296888281878178337303448562166771241047717584398969836670957312913054119887041158449273532128629495237793206991159034633290575319904995762059)

# Exchange

g^pk mod p

In [ ]:
pk = pow(g, sk, p)

# Encrypt

In [ ]:
def encrypt(plain_text, password):
    # generate a random salt
    salt = get_random_bytes(AES.block_size)

    # use the Scrypt KDF to get a private key from the password
    private_key = hashlib.scrypt(
        password.encode(), salt=salt, n=2**14, r=8, p=1, dklen=32)

    # create cipher config
    cipher_config = AES.new(private_key, AES.MODE_GCM)

    # return a dictionary with the encrypted text
    cipher_text, tag = cipher_config.encrypt_and_digest(
        bytes(plain_text, 'utf-8'))
    return {
        'cipher_text': b64encode(cipher_text).decode('utf-8'),
        'salt': b64encode(salt).decode('utf-8'),
        'nonce': b64encode(cipher_config.nonce).decode('utf-8'),
        'tag': b64encode(tag).decode('utf-8')
    }

# Decrypt


In [ ]:
def decrypt(data, password):
    # decode the dictionary entries from base64

    salt = b64decode(data['salt'])
    cipher_text = b64decode(data['cipher_text'])
    nonce = b64decode(data['nonce'])
    tag = b64decode(data['tag'])

    # generate the private key from the password and salt
    private_key = hashlib.scrypt(
        password.encode(), salt=salt, n=2**14, r=8, p=1, dklen=32)

    # create the cipher config
    cipher = AES.new(private_key, AES.MODE_GCM, nonce=nonce)

    # decrypt the cipher text
    decrypted = cipher.decrypt_and_verify(cipher_text, tag)

    return decrypted